# Chapter04: Naive Bayes

基于概率论的分类方法 —— 朴素贝叶斯（Naive Bayes）

## 朴素贝叶斯的特征

- 优点：在数据较少（n=？）的情况下仍然有效，可以处理多个类别的分类问题
- 缺点：对于输入数据的准备方式敏感

使用贝叶斯定理计算条件概率，比较输入值在不同类别中的概率大小，选取最大概率值作为其类别。

贝叶斯定理计算条件概率公式为：

${P(c|x)=\frac{p(x|c)p(c)}{p(x)}}$ 

其中，c为类别，x为特征

对于某一个特征向量X，如果 ${P(c1|X)}$ > ${P(c2|X)}$, 那X属于类别1；反之，属于类别2.

应用这一规则，我们可以构建朴素贝叶斯分类器。

接下来，构建朴素贝叶斯分类器，对博文进行分类。

载入本节的脚本 `bayes`, 使用其中的函数 `bayes.loadDataSet()` 创建从多个博文中抓取的词条保存在列表 `listOPosts` 中， 并创建**人工**标签 `listClasses` 保存了每篇博文是否为侮辱性博文（0代表正常言论，1代表侮辱性博文）。 再使用 `bayes.createVocabList(listOPosts)` 创建词汇表保存当前博文列表中所有的唯一词条。

构建数据：

- `postingList`: 列表，保存了每个博文中的词条
- `classVec`: 向量，人工对每篇博文的标注，0代表正常言论，1代表侮辱性博文

```python
def loadDataSet():
    postingList=[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                 ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                 ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                 ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                 ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                 ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec = [0,1,0,1,0,1]    #1 is abusive, 0 not
    return postingList,classVec
```


In [5]:
import bayes
listOPosts, listClasses = bayes.loadDataSet()
myVocabList = bayes.createVocabList(listOPosts)
myVocabList

['mr', 'problems', 'ate', 'how', 'not', 'flea', 'help', 'park', 'steak', 'quit', 'him', 'posting', 'so', 'love', 'garbage', 'buying', 'worthless', 'please', 'cute', 'has', 'stop', 'dog', 'take', 'stupid', 'maybe', 'dalmation', 'I', 'licks', 'is', 'my', 'food', 'to']

`bayes.setOfWords2Vec` 查询博文，例如第一篇博文 `listOPosts[0]` 的词条有哪些出现在了词汇表中。

函数返回结果为0/1值分别表示词汇表中的词没有或有被该篇博文用到。

这就相当于将字符值转换为数值。

In [6]:
bayes.setOfWords2Vec(myVocabList, listOPosts[0])

[0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]

这里使用循环将每一篇博文用到的词条在词汇表中的出现情况添加到矩阵中作为训练集

In [7]:
from importlib import reload
reload(bayes)
listOPosts, listClasses = bayes.loadDataSet()
myVocabList = bayes.createVocabList(listOPosts)
# create training dataset
trainMat = []
for postinDoc in listOPosts:
    trainMat.append(bayes.setOfWords2Vec(myVocabList, postinDoc))
    
trainMat

[[0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0]]

使用 `bayes.trainNB0(trainMat, listClasses)` 训练贝叶斯分类器，该函数进行以下操作：

- `pAbusive = sum(trainCategory)/float(numTrainDocs) #1 is abusive` 统计输入的博文有多少侮辱性博文

- 通过标签 `listClasses` 判断博文的类别，分别统计正常博文和侮辱性博文当中各词条出现的频率

- 返回侮辱性博文的概率，以及正常和侮辱博文中词条出现频率的对数


In [8]:
p0V,p1V,pAb = bayes.trainNB0(trainMat, listClasses)
#the probability of abusive posts in all posts
pAb

0.5

In [9]:
#词汇表中的词条在正常博文中出现的概率，经过log转换
p0V

array([-2.56494936, -2.56494936, -2.56494936, -2.56494936, -3.25809654,
       -2.56494936, -2.56494936, -3.25809654, -2.56494936, -3.25809654,
       -2.15948425, -3.25809654, -2.56494936, -2.56494936, -3.25809654,
       -3.25809654, -3.25809654, -2.56494936, -2.56494936, -2.56494936,
       -2.56494936, -2.56494936, -3.25809654, -3.25809654, -3.25809654,
       -2.56494936, -2.56494936, -2.56494936, -2.56494936, -1.87180218,
       -3.25809654, -2.56494936])

In [10]:
#词汇表中的词条在侮辱性博文中出现的概率，经过log转换 
p1V

array([-3.04452244, -3.04452244, -3.04452244, -3.04452244, -2.35137526,
       -3.04452244, -3.04452244, -2.35137526, -3.04452244, -2.35137526,
       -2.35137526, -2.35137526, -3.04452244, -3.04452244, -2.35137526,
       -2.35137526, -1.94591015, -3.04452244, -3.04452244, -3.04452244,
       -2.35137526, -1.94591015, -2.35137526, -1.65822808, -2.35137526,
       -3.04452244, -3.04452244, -3.04452244, -3.04452244, -3.04452244,
       -2.35137526, -2.35137526])

随后，使用 `bayes.classifyNB()` 判断向量计算在不同类别的概率，并根据概率判断其所属类别

```python
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    p1 = sum(vec2Classify * p1Vec) + log(pClass1)    #element-wise mult
    p0 = sum(vec2Classify * p0Vec) + log(1.0 - pClass1)
    if p1 > p0:
        return 1
    else: 
        return 0
```

使用 `bayes.testingNB()` 封装分类器的所有操作，并输入两篇文章中包含的词条向量进行分类

- `testEntry = ['love', 'my', 'dalmation']`; `testEntry = ['stupid', 'garbage']`


In [11]:
reload(bayes)
bayes.testingNB()

['love', 'my', 'dalmation'] classified as:  0
['stupid', 'garbage'] classified as:  1


上面我们只是考虑了词条出现与否，进一步我们考虑词条出现的频率。

添加函数 `bagOfWords2VecMN()`

In [12]:
reload(bayes)
listOPosts, listClasses = bayes.loadDataSet()
myVocabList = bayes.createVocabList(listOPosts)
listOPosts[0].extend(["dog", "dog"])
listOPosts[0]

['my', 'dog', 'has', 'flea', 'problems', 'help', 'please', 'dog', 'dog']

In [14]:
bayes.bagOfWords2VecMN(myVocabList, listOPosts[0])

[0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 3, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]

# 示例：使用朴素贝叶斯过滤垃圾邮件

该示例的步骤为：
1. 收集数据：书中的邮件例子
2. 准备数据：将文本文件解析为词条向量
3. 分析数据：检查词条确保解析的正契约型
4. 训练算法：使用之前构建的 `trainNB0()`
5. 测试算法：使用 `classifyNB()`，并构建测试函数计算文档集的错误率
6. 使用算法：构建一个完整的程序 `spamTest()` 对一组文档进行分类，并输出错分的文档和错误率到屏幕。

这里使用书中的邮件示例数据，数据分为了正常邮件(`email/ham/`)和垃圾邮件(`email/spam/`)各25封。

- 正常邮件
![image.png](./img/img1.png)

- 垃圾邮件
![image-2.png](./img/img2.png)



运行 `bayes.spamTest()` 时候报错
- 一个原因是切分字符的函数 `textParse()` 使用的正则匹配模式在原本的代码为 `listOfTokens = re.split(r'\W*', bigString)` 这样就会在对所有字符进行切分，而函数返回的字符长度必须大于2，这就导致空列表的返回；应该修改为 `listOfTokens = re.split(r'\W+', bigString)` 这样就只会在非单词字符处切分

- 还有另外一个错误如下：

    ![image.png](./img/img3.png)

修改encoding也不能解决这个问题，最后在这篇博文（https://blog.csdn.net/Dillon2015/article/details/52987568 ）中看到说是因为文件中有无法识别的字符才导致的问题。之后，我就查到"email/ham/23.txt" 这里有个问号，删掉之后就可以运行了

![image-2.png](./img/img4.png)

- 最后，还需要将95行的 `trainingSet = range(50)` 换为 `trainingSet = list(range(50))` （python3）

In [23]:
reload(bayes)
bayes.spamTest()

classification error ['benoit', 'mandelbrot', '1924', '2010', 'benoit', 'mandelbrot', '1924', '2010', 'wilmott', 'team', 'benoit', 'mandelbrot', 'the', 'mathematician', 'the', 'father', 'fractal', 'mathematics', 'and', 'advocate', 'more', 'sophisticated', 'modelling', 'quantitative', 'finance', 'died', '14th', 'october', '2010', 'aged', 'wilmott', 'magazine', 'has', 'often', 'featured', 'mandelbrot', 'his', 'ideas', 'and', 'the', 'work', 'others', 'inspired', 'his', 'fundamental', 'insights', 'you', 'must', 'logged', 'view', 'these', 'articles', 'from', 'past', 'issues', 'wilmott', 'magazine']
the error rate is:  0.1


这里要注意的是 `bayes.spamTest()` 返回的结果每次都可能是不同的，这是由于我们是随机抽取10个文件作为测试集。如果想评估分类器效果，可以多次运行该分类器，然后取错误率的平均值作为分类器的错误率

还有一个“使用朴素贝叶斯分类器从个人广告中获取区域倾向”的示例就先不展开了。

# 小结

以上还只是二分类的问题，多分类情况又该如何应用？